# TextSketch
Este _notebook_ implementa a DSL TextSketch, como descrita no [README](https://github.com/gui-clone/TextSketch).

Um **ponto** (`point`) pode ser descrito num espaço bidimensional. Apesar de ser definido aqui, ele é utilizado apenas para a implementação interna, não fazendo parte da interface da DSL.

In [1]:
(define-syntax point
  (syntax-rules ()
    ((point (a b))
     (format #f "~A ~A" a b))))

A **curva de bézier** (`bezier`) é o principal primitivo da nossa DSL. Com três pontos — inicial, final e de controle, criamos uma curva quadrática. Com a composição dessas curvas, podemos implementar tanto segmentos de reta quanto outras curvas arbitrárias.

Ela é implementada utilizando a diretiva `path` do SVG.

In [2]:
(define-syntax bezier
  (syntax-rules ()
    ((bezier sPT ePT cPT)
     `(path (d . ,(string-append
                   "M "
                   (point sPT)
                   " Q "
                   (point cPT)
                   " "
                   (point ePT)))
            (stroke . "black")
            (fill . "none")))))

Uma **forma** (`form`) compõe diversas curvas de bézier numa única diretiva `path` — por isso, seu valor de retorno tem o mesmo tipo de `bezier`.

In [ ]:
(define-syntax form
  (syntax-rules ()
    ((form shape ...) ; lista em define-syntax
     (let ((get_d (lambda (curve) (cdr (assq 'd (cdr curve))))))
       (newD (map get_d (list shape ...))))
     `(path (d . ,(string-join newD " "))
            (stroke . "black")
            (fill . "none")))))

Uma **união** (`union`) compõe diversos elementos num único `group` do SVG, permitindo a composição de diferentes formas numa única unidade.

In [4]:
(define-syntax union
  (syntax-rules ()
    ((union shape ...)
     (let* ((get_shape (lambda (x) (if (eq? 'group (car x)) (cadr x) x)))
            (temp_s (map get_shape (list shape ...)))
            (unite (lambda (x)
                     (if (and (list? x) (list? (car x)) (eq? 'path (caar temp_s)))
                         x
                         (list x))))
            (shapes (apply append (map unite temp_s))))
       `(group (,@shapes))))))

O macro `fill` adiciona uma cor de preenchimento à uma forma, sem modificar nenhuma outra propriedade; retornando a nova forma.

In [5]:
(define-syntax fill
  (syntax-rules ()
    ((fill color shape)
     (let ((newD (cdr (assq 'd (cdr shape))))
           (newStroke (cdr (assq 'stroke (cdr shape))))
           (newColor (format #f "~A" (quote color))))
       `(path (d . ,newD)
              (stroke . ,newStroke)
              (fill . ,newColor))))))

O macro `new-panel` retorna uma função que pode ser utilizada para converter elementos em uma estrutura SVG, num plano com a largura e altura indicados.

In [6]:
(define-syntax new-panel
  (syntax-rules ()
    ((new-panel width height)
     (lambda (shape)
       (let* ((type (car shape))
              (content (cdr shape))
              (toSVG (lambda (cont)
                       (let ((newD (cdr (assq 'd cont)) )
                             (newStroke (cdr (assq 'stroke cont)) )
                             (newColor (cdr (assq 'fill cont)) ))

                         `(<path
                           d= ,(format #f "~A" newD )
                           stroke= ,(format #f "~A" newStroke)
                           fill= ,(format #f "~A" newColor)
                           stroke-width="2"
                           />)))))
         (if (eq? 'path type)
             (append
              `(<svg
                width="300"
                height="300"
                viewBox= ,(format #f "0 0 ~A ~A" width height) >)
              (toSVG content)
              '(</svg>))
             (append
              `(<svg
                width="300"
                height="300"
                viewBox= ,(format #f "0 0 ~A ~A" width height) >)
              (map (lambda (x) (toSVG (cdr x))) (car content))
              '(</svg>))))))))

O macro `defineSVG` possibilita a construção de formas/grupos com diferentes argumentos, possibilitando a criação automatizada de diferentes variações de um mesmo objeto. Ele retorna outro macro, que pode ser chamado com os argumentos correspondentes para produzir a forma final.

In [7]:
(define-syntax defineSVG
  (syntax-rules ()
    ((defineSVG name (args ...)
       template)
     (define-syntax name
       (syntax-rules ()
         ((name args ...) template))))))